In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./stackoverflow-data/survey_results_public.csv')
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [3]:
df.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [4]:
df.shape

(88883, 85)

In [59]:
df['DevEnviron'].value_counts()

Visual Studio Code                                                                                     5036
Visual Studio;Visual Studio Code                                                                       3840
Notepad++;Visual Studio;Visual Studio Code                                                             3248
Vim                                                                                                    2224
Notepad++;Visual Studio                                                                                1968
                                                                                                       ... 
Android Studio;Coda;NetBeans;Notepad++;Sublime Text;Visual Studio;Xcode                                   1
Eclipse;NetBeans;Notepad++;PHPStorm;Sublime Text;TextMate;Vim;Visual Studio;Visual Studio Code;Zend       1
NetBeans;PHPStorm;Vim;Visual Studio                                                                       1
Atom;NetBeans;Vim;Xcode     

In [60]:
df['DevEnviron'].value_counts().shape[0]

7716

In [87]:
data_plot = df['DevEnviron'].str.cat(sep = ';')
data = pd.Series(data_plot.split(';'))
print(data.value_counts())
# compare against all records in original df that are not null
print("vim is ", data.value_counts()[4] / df[pd.notna(df['DevEnviron'])].shape[0])
# note that these are not exclusive, respondants were able to pick more that one ide
print("vscode is ", data.value_counts()[0] / df[pd.notna(df['DevEnviron'])].shape[0])


Visual Studio Code    44311
Visual Studio         27490
Notepad++             26621
IntelliJ              22166
Vim                   22163
Sublime Text          20424
Android Studio        14787
Eclipse               12591
PyCharm               11724
Atom                  11636
IPython / Jupyter      8317
Xcode                  8239
PHPStorm               6645
NetBeans               5121
Emacs                  3922
RStudio                2940
RubyMine               1202
TextMate                805
Coda                    578
Komodo                  368
Zend                    307
Light Table             178
dtype: int64
vim is  0.2538222797393406
vscode is  0.5074727716252276


In [22]:
# df[df.dtypes != 'object']
categorical_columns = df.select_dtypes(include=['object']).copy()
# categorical_columns = df.select_dtypes(include=['object'])
categorical_columns.shape

(88883, 79)

In [53]:
na_category_totals = categorical_columns.isna().sum()
total_category_rows = categorical_columns.shape[0]
percentage_cutoff = 0.10
amount = categorical_columns.columns[na_category_totals / total_category_rows > percentage_cutoff].shape[0]
print("%s categories out of %s (%s) have a null value percentage over %i percent" %(amount, total_category_rows, amount / total_category_rows, percentage_cutoff * 100))

39 categories out of 88883 (0.00043877906911332876) have a null value percentage over 10 percent


In [ ]:
# dummy categories
# dummy_columns = pd.get_dummies(categorical_columns['col1'],dummy_na=True)

In [56]:
from pandas.api.types import CategoricalDtype

s = pd.Series(["a", "bc", "c", "d"])
cat_type = CategoricalDtype(categories=["b", "c", "d"], ordered=True)
print(s)
s_cat = s.astype(cat_type)
print(s_cat)


0     a
1    bc
2     c
3     d
dtype: object
0    NaN
1    NaN
2      c
3      d
dtype: category
Categories (3, object): [b < c < d]


In [94]:
ide = df['DevEnviron']
print(ide.head())
print(ide.shape)

0                      IntelliJ;Notepad++;PyCharm
1                                    Atom;PyCharm
2                          Vim;Visual Studio Code
3    Eclipse;Vim;Visual Studio;Visual Studio Code
4                          IntelliJ;Notepad++;Vim
Name: DevEnviron, dtype: object
(88883,)


In [95]:
ide.str.split(';', expand=True)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,IntelliJ,Notepad++,PyCharm,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Atom,PyCharm,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Vim,Visual Studio Code,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Eclipse,Vim,Visual Studio,Visual Studio Code,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,IntelliJ,Notepad++,Vim,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88878,Atom,Visual Studio,Visual Studio Code,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
88879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
ide[ide.notna()].shape

(87317,)

In [105]:
ide_dropped = ide.dropna()

In [106]:
ide_dropped.str.split(';', expand=True)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,IntelliJ,Notepad++,PyCharm,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Atom,PyCharm,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Vim,Visual Studio Code,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Eclipse,Vim,Visual Studio,Visual Studio Code,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,IntelliJ,Notepad++,Vim,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88875,Android Studio,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
88876,Atom,Visual Studio Code,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
88877,Vim,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
88878,Atom,Visual Studio,Visual Studio Code,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
